In [1]:
import sys 
import os

import pandas as pd
import pymoab
import pydagmc
from pymoab import core
from pymoab import types

from pymoab.tag import Tag
import numpy as np

In [2]:
pymoab.__version__

'unknown'

In [3]:
mb = core.Core()

In [4]:
model = pydagmc.Model("./rtttest_mcnp_mesh.rtt")
mb = model.mb
rs = mb.get_root_set()
hdle = mb.get_entities_by_handle(rs)

/usr/local/lib/python3.12/dist-packages/pydagmc/dagnav.py:343: UserWarning: Assigned geom_dimension=4 to group 'graveyard_comp'.
  warn(f"Assigned geom_dimension={self._geom_dimension} to {identifier}.")


In [5]:
def GetMaterialList(moabcore):
    root_set = mb.get_root_set()
    
    category_tg_name = mb.tag_get_handle("CATEGORY", 32, types.MB_TYPE_OPAQUE, False)
    group_handle = mb.get_entities_by_type_and_tag(root_set, types.MBENTITYSET, category_tg_name, np.array(['Group']))
 
    name_tg_name = mb.tag_get_handle("NAME", 32, types.MB_TYPE_OPAQUE, False)
    mat_list = []
    for i in group_handle:
        try:
            mat_name = mb.tag_get_data(name_tg_name, i)[0][0]
            if mat_name[0:3] == "mat" :
                mat_list.append(mat_name)
        except RuntimeError:
            continue
    return mat_list

def GetMaterialHandle(moabcore):
    root_set = mb.get_root_set()
    hdle_list = mb.get_entities_by_handle(root_set)
    tg_name = mb.tag_get_handle("NAME", 32, types.MB_TYPE_OPAQUE, False)
    mat_hld = []
    for i in hdle:
        try:
            mat_name = mb.tag_get_data(tg_name, i)[0][0]
            if mat_name[0:3] == "mat" :
                mat_hld.append(i)
        except RuntimeError:
            continue
    return mat_hld

def GetMaterialAssignment(moabcore):
    mat_assignment = []
    root_set = mb.get_root_set()
    mat_list = GetMaterialList(moabcore)
    tg_name = mb.tag_get_handle("NAME", 32, types.MB_TYPE_OPAQUE, False)
    for i in range(len(mat_list)):
        mat_hdle = mb.get_entities_by_type_and_tag(root_set, types.MBENTITYSET, tg_name, np.array([mat_list[i]]))
        for hld in mat_hdle:
            assignent_mat_tmp = []
            for vol_hld in mb.get_entities_by_handle(hld):
                assignent_mat_tmp.append(vol_hld)
            mat_assignment.append(assignent_mat_tmp)
    return mat_assignment



def GetVolumeHandle(moabcore):
    vol_handle = []
    root_set = mb.get_root_set()
    tg_name = mb.tag_get_handle("CATEGORY", 32, types.MB_TYPE_OPAQUE, False)
    volume_handle = mb.get_entities_by_type_and_tag(root_set, types.MBENTITYSET, tg_name, np.array(['Volume']))
    for hld in volume_handle:
        vol_handle.append(hld)
    return vol_handle

def GetVolumeMaterial(moabcore):
    rtn = []
    root_set = mb.get_root_set()
    mat_list = GetMaterialList(moabcore)
    tg_name = mb.tag_get_handle("NAME", 32, types.MB_TYPE_OPAQUE, False)
    for i in range(len(mat_list)):
        mat_hdle = mb.get_entities_by_type_and_tag(root_set, types.MBENTITYSET, tg_name, np.array([mat_list[i]]))
        for hld in mat_hdle:
            for hld_2 in mb.get_entities_by_handle(hld):
                print(mb.tag_get_data(tg, hld), " ", hld_2, " ", mb.tag_get_data(tg, hld_2))
                
def GetVolumeId(moabcore):
    rtn = []
    volume_identities = []
    vol_hdl = GetVolumeHandle(moabcore)
    tg_name = mb.tag_get_handle("GLOBAL_ID", 1, types.MB_TYPE_INTEGER, False)
    for i in vol_hdl:
        try:
            volume_identities.append(mb.tag_get_data(tg_name, i)[0][0])
        except RuntimeError:
            continue
    return volume_identities

def GetMaterialId(moabcore):
    rtn = []
    volume_identities = []
    vol_hdl = GetMaterialHandle(moabcore)
    tg_name = mb.tag_get_handle("GLOBAL_ID", 1, types.MB_TYPE_INTEGER, False)
    for i in vol_hdl:
        try:
            volume_identities.append(mb.tag_get_data(tg_name, i)[0][0])
        except RuntimeError:
            continue
    return volume_identities


def GetVolumeMaterialAssignment_handle(moabcore):
    mat_assigment = GetMaterialAssignment(moabcore)
    volume_handle = GetVolumeHandle(moabcore)
    mat_handle = GetMaterialHandle(moabcore)

    mat_in_vol = []
    for vol_hdl in volume_handle:
        for i, asgmts in enumerate(mat_assignment):
            if vol_hdl in asgmts:
                mat_in_vol.append(mat_handle[i])
    return mat_in_vol

def GetVolumeMaterialAssignment_name(moabcore):
    mat_assigment = GetMaterialAssignment(moabcore)
    volume_handle = GetVolumeHandle(moabcore)
    mat_name = GetMaterialList(moabcore)

    mat_in_vol = []
    for vol_hdl in volume_handle:
        for i, asgmts in enumerate(mat_assignment):
            if vol_hdl in asgmts:
                mat_in_vol.append(mat_name[i])
    return mat_in_vol




In [6]:
mb.write_file("rtttest_mcnp_mesh_out.h5")

Using default writer WriteHDF5 for file rtttest_mcnp_mesh_out.h5 


In [7]:
mat_list = GetMaterialList(mb)
mat_hdl = GetMaterialHandle(mb)
mat_assignment = GetMaterialAssignment(mb)
vol_hdl = GetVolumeHandle(mb)
vol_id = GetVolumeId(mb)
mat_id = GetMaterialId(mb)
mat_vol = GetVolumeMaterialAssignment_handle(mb)
mat_vol_name = GetVolumeMaterialAssignment_name(mb)



index = range(0, len(vol_hdl))
columns=['ID', 'Handle', 'Material name', 'Material handle']
pdf = pd.DataFrame(index=index, columns=columns)
for x in range(len(vol_hdl)):
    pdf.loc[[x], 'ID'] = vol_id[x]
    pdf.loc[[x], 'Handle'] = vol_hdl[x]
    pdf.loc[[x], 'Handle'] = vol_hdl[x]
    pdf.loc[[x], 'Material handle'] = mat_vol[x]
    pdf.loc[[x], 'Material name'] = mat_vol_name[x]


In [9]:
pdf

,ID,Handle,Material name,Material handle
0,1,12682136550675316758,mat:Concrete,12682136550675316763
1,2,12682136550675316759,mat:VOID,12682136550675316764
2,3,12682136550675316760,mat:Air,12682136550675316765


In [8]:
mat_list = GetMaterialList(mb)
print("Material list: ", mat_list)
mat_hdl = GetMaterialHandle(mb)
print("Material handle: ", mat_hdl)
mat_assignment = GetMaterialAssignment(mb)
print("Material assignement: ", mat_assignment)
vol_hdl = GetVolumeHandle(mb)
print("Volume handle: ", vol_hdl)
vol_id = GetVolumeId(mb)
print("Volume ID: ", vol_id)
mat_id = GetMaterialId(mb)
print("Material ID: ", mat_id)
mat_vol = GetVolumeMaterialAssignment_handle(mb)
print("VolMat: ", mat_vol)
mat_vol_name = GetVolumeMaterialAssignment_name(mb)
print("VolMat Name: ", mat_vol_name)






Material list:  [np.str_('mat:Concrete'), np.str_('mat:VOID'), np.str_('mat:Air')]
Material handle:  [np.uint64(12682136550675316763), np.uint64(12682136550675316764), np.uint64(12682136550675316765)]
Material assignement:  [[np.uint64(12682136550675316758)], [np.uint64(12682136550675316759)], [np.uint64(12682136550675316760)]]
Volume handle:  [np.uint64(12682136550675316758), np.uint64(12682136550675316759), np.uint64(12682136550675316760)]
Volume ID:  [np.int32(1), np.int32(2), np.int32(3)]
Material ID:  [np.int32(1), np.int32(3), np.int32(2)]
VolMat:  [np.uint64(12682136550675316763), np.uint64(12682136550675316764), np.uint64(12682136550675316765)]
VolMat Name:  [np.str_('mat:Concrete'), np.str_('mat:VOID'), np.str_('mat:Air')]
